In [2]:
import pickle
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import networkx as nx
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import time
from Bio import SeqIO

In [4]:
with open('GAG.fasta', 'r') as f:
    seq = list(SeqIO.parse(f, 'fasta'))

In [3]:
def timeit(func):
    def wrapper():
        start = time.time()
        func()
        print("calc time={}".format(time.time() - start))
    return wrapper

In [7]:
# Levenshtein distance
import Levenshtein as lvn

def distance(args):
    a, b = args
    return lvn.distance(str(a),str(b))

In [5]:
# Needleman Wunshe distance
from Bio import pairwise2

def distance(args):
    a, b = args
    return pairwise2.align.globalxx(a,b)

In [10]:
# Multiprocessing calculation
from multiprocessing import Pool

@timeit
def calca_e2e():
    data = []
    for i in range(len(seq)):
        for j in range(i, len(seq)):
            a, b = seq[i].seq, seq[j].seq
            data.append((a,b))
    with Pool(8) as p:
        cdata = p.map(distance, data)

    with open('linear_data.pkl', 'wb') as f:
        pickle.dump(cdata, f)
        
calca_e2e()

calc time=137.63756895065308


In [5]:
# linear data to matrix

with open('linear_data.pkl', 'rb') as f:
    cdata = pickle.load(f)

matrix = [[0]*len(seq) for _ in range(len(seq))]
icdata = iter(cdata)
for i in range(len(seq)):
    for j in range(i, len(seq)):
        score = next(icdata)
        matrix[i][j] = score
        matrix[j][i] = score

with open('matrix.pkl', 'wb') as f:
    pickle.dump(matrix, f)